In [16]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [24]:
# read data
convolution = pd.read_csv('data/convolution_benchmark_1565254576148.csv', index_col='env')
fft = pd.read_csv('data/fft_benchmark_1565254576148.csv', index_col='env')

In [27]:
convolution.head()

,10,20,30,40,50,60,70,80,90,100,...,910,920,930,940,950,960,970,980,990,1000
env,,,,,,,,,,,,,,,,,,,,,
jvm,8.333333e+06,8.333333e+06,5.555556e+06,5.555556e+06,4.545455e+06,4.545455e+06,4.166667e+06,3.333333e+06,2.777778e+06,3.571429e+06,...,312500.000000,310559.006211,306748.466258,303030.30303,299401.197605,373134.328358,303030.303030,357142.857143,289017.341040,297619.047619
ndk,3.333333e+06,1.851852e+06,1.351351e+06,1.470588e+06,1.388889e+06,1.190476e+06,1.020408e+06,9.090909e+05,7.812500e+05,6.944444e+05,...,82918.739635,81967.213115,81433.224756,80256.82183,79744.816587,78492.935636,77760.497667,76452.599388,76452.599388,75528.700906


In [11]:
fft.head()

,1024,2048,3072,4096,5120,6144,7168,8192,9216,10240,...,41984,43008,44032,45056,46080,47104,48128,49152,50176,51200
env,,,,,,,,,,,,,,,,,,,,,
jvm,2083.333333,4761.904762,1694.915254,518.134715,1612.903226,2083.333333,1408.450704,2857.142857,1754.385965,1538.461538,...,161.290323,258.397933,149.925037,221.238938,248.138958,176.99115,129.366106,242.718447,167.22408,237.529691
ndk,25000.000000,14285.714286,8333.333333,5555.555556,4761.904762,4166.666667,3448.275862,2941.176471,2439.024390,2040.816327,...,317.460317,505.050505,322.580645,427.350427,460.829493,359.71223,279.329609,440.528634,413.22314,338.983051


# Convolution (FIR filter)

In [36]:
iplot({
    'data': [
        go.Scatter(**dict(
            x = convolution.T.index,
            y = convolution.T[env],
            mode = 'lines', # markers
            name = env.upper(),
        )) for env in ['jvm', 'ndk']
    ],
    'layout': go.Layout(title='Convolution Benchmark', xaxis=dict(title='filter length'), yaxis=dict(title='samples / s'))
})

# FFT

In [37]:
iplot({
    'data': [
        go.Scatter(**dict(
            x = fft.T.index,
            y = fft.T[env],
            mode = 'lines', # markers
            name = env.upper(),
        )) for env in ['jvm', 'ndk']
    ],
    'layout': go.Layout(title='FFT Benchmark', xaxis=dict(title='FFT width'), yaxis=dict(title='FFTs / s'))
})